In [ ]:
def rect_to_bb(rect):

    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):

    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
    	coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [ ]:
def save(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/drive/My Drive/dsproj/ads_proj_data/ads_proj_data/shape_predictor_68_face_landmarks.dat')


    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    rects = detector(gray, 1)
    if len(rects) > 1:
    	print( "ERROR: more than one face detected")
    	return
    if len(rects) < 1:
    	print( "ERROR: no faces detected")
    	return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        clone = gray.copy()

        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)
        print('cropped/cropped/' + write_img_path)
        cv2.imwrite('cropped/cropped/' + write_img_path, roi)

In [ ]:
import shutil

def crop():

    i = 1
    for person_ID in people:
        if not os.path.exists('cropped/cropped/' + person_ID ):
            os.mkdir('cropped/cropped/' + person_ID + '/')

        for dt in dts:
            if not os.path.exists('cropped/cropped/' + person_ID + '/' + dt):
                os.mkdir('cropped/cropped/' + person_ID + '/' + dt)

            for phrase_ID in folder_enum:
                if not os.path.exists('cropped/cropped/' + person_ID + '/' + dt + '/' + phrase_ID):

                    os.mkdir('cropped/cropped/' + person_ID + '/' + dt + '/' + phrase_ID)

                for instance_ID in instances:

                    directory = '/content/drive/My Drive/dsproj/ads_proj_data/ads_proj_data' + '/' + person_ID + '/' + dt + '/' + phrase_ID + '/' + instance_ID + '/'
                    dir_temp = person_ID + '/' + dt + '/' + phrase_ID + '/' + instance_ID + '/'

                    filelist = os.listdir(directory)
                    if not os.path.exists('cropped/cropped/' + person_ID + '/' + dt + '/' + phrase_ID + '/' + instance_ID):
                        os.mkdir('cropped/cropped/' + person_ID + '/' + dt + '/' + phrase_ID + '/' + instance_ID)

                        for img_name in filelist:
                            if img_name.startswith('color'):
                                image = imageio.imread(directory + '' + img_name)
                                save(image, directory + '' + img_name,
                                                    dir_temp + '' + img_name, img_name)

    print(f'Iteration : {i}')
    i += 1
    shutil.rmtree('cropped')

In [ ]:
import time
#os.mkdir('cropped')
times = 0
for _ in range(7):
    t1 = time.time()
    crop()
    t2 = time.time()
    times += (t2 - t1)

print("Average time over 7 iterations : ", times/7)